# Создание датасета для обучения предиктивной модели
Этот Notebook демонстрирует процесс создания подмножества датасета из DocBank (4000 изображений) для классификации искажений (clean, bad_print, brightness_contrast, pixelation).

Изображения делятся на классы, применяются эффекты, сохраняются в train/val.

Исходный датасет можно скачать тут - https://doc-analysis.github.io/docbank-page/index.html

## Шаг 1: Импорт библиотек
Импортируем библиотеки для работы с изображениями, путями и эффектами.

In [ ]:
import os
import cv2
import numpy as np
from pathlib import Path
import random
from tqdm import tqdm
from effects import apply_bad_print_effect, apply_aging_effect, apply_brightness_contrast, apply_pixelation

## Шаг 2: Настройка путей
Определяем пути к исходному датасету и выходной директории.

In [ ]:
dataset_path = Path("D:/DocBank_500K_ori_img")
output_path = Path("D:/DocBank_Subset_Prediction_4000")
output_path.mkdir(exist_ok=True)

## Шаг 3: Определение классов
Список классов для датасета.

In [ ]:
classes = ['clean', 'bad_print', 'brightness_contrast', 'pixelation']

## Шаг 4: Выбор случайных изображений
Выбираем 4000 случайных JPG-файлов из датасета.

In [ ]:
image_files = list(dataset_path.glob("*.jpg"))
random.shuffle(image_files)
selected_files = image_files[:4000]

## Шаг 5: Разделение на классы
Делим изображения на 4 класса по 1000 штук.

In [ ]:
images_per_class = 4000 // 4
tasks = {
    'clean': selected_files[:images_per_class],
    'bad_print': selected_files[images_per_class:2*images_per_class],
    'brightness_contrast': selected_files[2*images_per_class:3*images_per_class],
    'pixelation': selected_files[3*images_per_class:]
}

## Шаг 6: Обработка изображений
Для каждого класса создаём подпапки train/val, применяем эффекты (для clean — оригинал), сохраняем в RGB.

In [ ]:
for cls, files in tasks.items():
    print(f"Обработка класса: {cls}")
    
    train_dir = output_path / "train" / cls
    val_dir = output_path / "val" / cls
    for p in [train_dir, val_dir]:
        p.mkdir(parents=True, exist_ok=True)
    
    train_size = int(0.8 * len(files))
    train_files = files[:train_size]
    val_files = files[train_size:]
    
    for img_path in tqdm(train_files, desc=f"{cls} - Train"):
        img = cv2.imread(str(img_path))
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if cls == 'bad_print':
            processed = apply_bad_print_effect(img)
            processed = apply_aging_effect(processed)
        elif cls == 'brightness_contrast':
            processed = apply_brightness_contrast(img)
        elif cls == 'pixelation':
            processed = apply_pixelation(img)
        else:
            processed = img
        
        save_path = train_dir / img_path.name
        cv2.imwrite(str(save_path), cv2.cvtColor(processed, cv2.COLOR_RGB2BGR))
    
    for img_path in tqdm(val_files, desc=f"{cls} - Val"):
        img = cv2.imread(str(img_path))
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        if cls == 'bad_print':
            processed = apply_bad_print_effect(img)
            processed = apply_aging_effect(processed)
        elif cls == 'brightness_contrast':
            processed = apply_brightness_contrast(img)
        elif cls == 'pixelation':
            processed = apply_pixelation(img)
        else:
            processed = img
        
        save_path = val_dir / img_path.name
        cv2.imwrite(str(save_path), cv2.cvtColor(processed, cv2.COLOR_RGB2BGR))